In [1]:
# install necessary packages 

!pip install wbdata
import wbdata # IF NECESSARY
import cufflinks as cf
cf.go_offline()
import pandas as pd
import numpy as np
from scipy import stats

## For Reviewers: 

For this project, we decided to focus on populations with exceptionally high rates of migration in the past 10 years, and related indicators associated with the WB's environmentally-focused dataset (which, importantly, only started being collected after 2014). This notebook will proceed as follows: 

1. Functions that will be used later on are defined. 

2. First, we need to figure out which countries are characteristic of this focus, so we
    - a) Load a dataframe with solely population and migration data
    - b) Calculate per capita information in order to proportionally compare migration statistics 
    - c) Sort and analyze this dataframe, isolate countries with the top ~ 20% of migration rates
    - d) Prepare this subpopulation data for our next steps.
    
3. Then, we load the necessary dataframe with the population statistics information we need to analyze, along with indicators we thought were relevant to this exploration. Then: 
    - a) Clean this dataframe, and append migration statistics
    - b) Write a function to retrieve the population statistics data 
    - c) Write a unit-test to perform on our function. 
    
Also, to note: currently, our figures (population pyramids) are *in the works* (they are challenging as we are trying to build population pyramids with multiple country inputs.) We decided not to include them in this code review, perhaps not to our benefit, but that is just where we are at right now! 

## FUNCTIONS

In [12]:
''' This function is used to clean the data, and in order to calculate per-capita statistics 
    we need specifically for isolating the high-migration population.'''

def migration_organizing(data1):
    
    # Make years ints instead of strings
    data1.reset_index(inplace=True)
    data1['date'] = data1['date'].astype(int)

    # index the table by country instead of year 
    data1.set_index(['country'],inplace=True)
    
    #takes all negative values, we want migration AWAY
    mig_table = data1[data1['Net Migration'] < 0] 
    
    #takes absolute value to get magnitude rather than negative
    mig_table['Net Migration'] = mig_table['Net Migration'].abs() 
    
    #creates new column, called 'Migration Per Capita'
    mig_table['Migration per Capita'] = mig_table['Net Migration']/mig_table['Total Population'] 
    
    #takes migration per capita and makes it a rate
    mig_table['Migration Rate (%)'] = mig_table['Migration per Capita']*100   
    
    # This line of code is initialized because we are interested in a population contingent 
    # on trends in the past ten years (rather than since '64). Additionally, we look at data 
    # from the ESG dataset, which only started being compiled in 2014'''
    
    mig_table = mig_table[mig_table['date'] > 2013] 

    return mig_table



''' This function is a manual way of creating a dataframe that is usually performed by 
    a "group_by" function. The reason for this distinction is because we used the group_by
    function, but were really struggling to perform table-operations on the group_by frame, 
    as that function creates a special type of dataframe. This method was much faster.'''

def setup_long(dataframe):
    
    #percentiles = {'values': [50, 75, 80, 90]}
    unique_country_indices = dataframe.index.unique()

    if not unique_country_indices.empty:
        
        # Create an empty DataFrame with columns
        results = pd.DataFrame(columns=['Net Migration', 'Migration Rate (%)'])

        for country_index in unique_country_indices:
            country_data = dataframe.loc[country_index]
            mig_net_avg = country_data['Net Migration'].mean()
            mig_percap_avg = country_data['Migration Rate (%)'].mean()

            # Append the computed averages to the results DataFrame
            results.loc[country_index] = [mig_net_avg, mig_percap_avg]
        
        # calculate the percentiles for the migration rates 
        results['Percentile Rank'] = results['Migration Rate (%)'].apply(
            lambda x: stats.percentileofscore(results['Migration Rate (%)'], x))

        return results 
    
    
    
''' This is the function that we use to retrieve population statistics, as outlined in the 
    [A] deliverables posted on Ed.'''


def population(year, sex, age_low, age_high, country_code):
    
    # reconstruct the strings for population codes associated 
    # as entered in "age_low" and "age_high" arguments
    
    if sex == "Male":
        column_names = {"SP.POP." + str(age_low)+str(age_high) + ".MA": sex}
    elif sex == "Female":
        column_names = {"SP.POP." + str(age_low)+str(age_high) + ".FE": sex}
    
    # construct new dataframe for function to index, isolating 
    # the country by the function's country-code argument
    pop_stats = wbdata.get_dataframe(column_names, country = country_code)
    
    # filter the table by the function's year' argument
    
    pop_stats = pop_stats.filter(like=str(year), axis=0)
    # return population number by indexing the function-generated 
    # dataframe by the function's 'sex' argument, and making it an integer

    return int(pop_stats[sex].iloc[0])



''' This function retrieves a dataframe for specific year, country, and indicators selected. 
    The function assumes that the argument pop_indicators has already been defined with a 
    relevant WBData code dictionary.'''

def population_dataframe(year, country_code, pop_indicators):
        
    pop_df = wbdata.get_dataframe(pop_indicators, country = country_code)
    
    # filter the table by the function's year' argument
    
    pop_df = pop_df.filter(like=str(year), axis=0)
    
    # return population dataframe by indexing  
    # by the function's 'sex' argument

    return pop_df

## FINDING HIGH-MIGRATION COUNTRIES

In [15]:
# call the WB dataframe 

indicators = {"SM.POP.NETM": "Net Migration", "SP.POP.TOTL": "Total Population"}

mig_pop_data = wbdata.get_dataframe(indicators)
mig_pop_data

Net Migration  Total Population
country                     date                                 
Africa Eastern and Southern 2022            NaN       720859132.0
                            2021      -179444.0       702977106.0
                            2020       -48955.0       685112979.0
                            2019      -187410.0       667242986.0
                            2018      -366105.0       649757148.0
...                                         ...               ...
Zimbabwe                    1964       -10064.0         4310332.0
                            1963        -9369.0         4177931.0
                            1962        -8931.0         4049778.0
                            1961        -8582.0         3925952.0
                            1960        -8256.0         3806310.0

[16758 rows x 2 columns]

In [11]:
# clean the data and calculate per-capita information for 
# migration with pre-defined organizing function 

mig_pop_cleaned = migration_organizing(mig_pop_data)
mig_pop_cleaned

,date,Net Migration,Total Population,Migration per Capita,Migration Rate (%)
country,,,,,
Africa Eastern and Southern,2021,179444.0,702977106.0,0.000255,0.025526
Africa Eastern and Southern,2020,48955.0,685112979.0,0.000071,0.007146
Africa Eastern and Southern,2019,187410.0,667242986.0,0.000281,0.028087
Africa Eastern and Southern,2018,366105.0,649757148.0,0.000563,0.056345
Africa Eastern and Southern,2017,343075.0,632746570.0,0.000542,0.054220
...,...,...,...,...,...
Zimbabwe,2018,59918.0,15052184.0,0.003981,0.398068
Zimbabwe,2017,59918.0,14751101.0,0.004062,0.406193
Zimbabwe,2016,59918.0,14452704.0,0.004146,0.414580


In [13]:
# get migration rate average for each country using 
# pre-defined function, sort from highest to lowest

avgs = setup_long(mig_pop_cleaned)
avgs = avgs.sort_values(by = ['Migration Rate (%)'], ascending = False)
avgs

,Net Migration,Migration Rate (%),Percentile Rank
Curacao,9592.000000,6.092441,100.000000
Marshall Islands,1901.000000,4.107915,99.481865
Qatar,97400.000000,3.543467,98.963731
Syrian Arab Republic,678143.500000,3.455758,98.445596
American Samoa,1638.500000,3.365898,97.927461
...,...,...,...
Late-demographic dividend,128646.000000,0.005552,2.590674
Ethiopia,6097.000000,0.005311,2.072539
Viet Nam,3503.875000,0.003739,1.554404
Madagascar,990.333333,0.003532,1.036269


In [14]:
# get dataframe with highest 10 percent migration rates

avgs1 = avgs[avgs['Percentile Rank'] > 90]
avgs1

,Net Migration,Migration Rate (%),Percentile Rank
Curacao,9592.000000,6.092441,100.000000
Marshall Islands,1901.000000,4.107915,99.481865
Qatar,97400.000000,3.543467,98.963731
Syrian Arab Republic,678143.500000,3.455758,98.445596
American Samoa,1638.500000,3.365898,97.927461
Lebanon,197743.142857,3.292441,97.409326
South Sudan,337661.571429,3.104315,96.891192
Kuwait,116678.750000,2.798996,96.373057
St. Martin (French part),909.875000,2.696374,95.854922
"Venezuela, RB",578053.125000,1.960295,95.336788


In [7]:
# Retrieve the index of these countries as a list of strings

country_array = avgs1.index.tolist()

# THIS is our selected "population"!! 

country_array

['Curacao',
 'Marshall Islands',
 'Qatar',
 'Syrian Arab Republic',
 'American Samoa',
 'Lebanon',
 'South Sudan',
 'Kuwait',
 'St. Martin (French part)',
 'Venezuela, RB',
 'Oman',
 'Tonga',
 'Saudi Arabia',
 'Kosovo',
 'Northern Mariana Islands',
 'Moldova',
 'Central African Republic',
 'Bahrain',
 'Guyana',
 'Samoa']

In [8]:
# build list of strings for WB country codes associated with each of these countries 
# could not iterate through the WB code list using the "wbdata.get_countries() function, but would like to" 

country_codes = ['CUW', 'MHL', 'QAT', 'SYR', 'ASM', 'LBN', 'SSD', 'KWT', 'MAF', 'VEN', 
    'OMN', 'TON', 'SAU', 'XKX', 'MNP', 'MDA', 'CAF', 'BHR', 'GUY', 'WSM', 
    'PRI', 'ERI', 'FJI', 'FSM', 'TUV', 'BIH', 'GUM', 'VCT', 'ARM', 'PSS', 
    'STP', 'ZAF', 'SWZ', 'NCL', 'SLV', 'BDI', 'PAK', 'KIR', 'GRL', 'DMA', 
    'LTU', 'NPL']

## Population Dataframe and Testing

In [62]:
# set up list of population and environmental indicators we are interested in

pop_env_indicators = {"SP.POP.TOTL": "Total Population", # first, pop stats
                      "SP.POP.TOTL.FE.IN": "Population, female",
                      "SP.POP.TOTL.MA.IN": "Population, male",
                      "SP.POP.0004.FE": "Population ages 00-04, female",
                      "SP.POP.0004.MA": "Population ages 00-04, male",
                      "SP.POP.0014.FE.IN": "Population ages 0-14, female",
                      "SP.POP.0014.MA.IN": "Population ages 0-14, male",
                      "SP.POP.0014.TO": "Population ages 0-14, total",
                      "SP.POP.0509.FE": "Population ages 05-09, female",
                      "SP.POP.0509.MA": "Population ages 05-09, male",
                      "SP.POP.1014.FE": "Population ages 10-14, female",
                      "SP.POP.1014.MA": "Population ages 10-14, male",
                      "SP.POP.1519.FE": "Population ages 15-19, female",
                      "SP.POP.1519.MA": "Population ages 15-19, male",
                      "SP.POP.1564.FE.IN": "Population ages 15-64, female",
                      "SP.POP.1564.MA.IN": "Population ages 15-64, male",
                      "SP.POP.1564.TO": "Population ages 15-64, total",
                      "SP.POP.2024.FE": "Population ages 20-24, female",
                      "SP.POP.2024.MA": "Population ages 20-24, male",
                      "SP.POP.2529.FE": "Population ages 25-29, female",
                      "SP.POP.2529.MA": "Population ages 25-29, male",
                      "SP.POP.3034.FE": "Population ages 30-34, female",
                      "SP.POP.3034.MA": "Population ages 30-34, male",
                      "SP.POP.3539.FE": "Population ages 35-39, female",
                      "SP.POP.3539.MA": "Population ages 35-39, male",
                      "SP.POP.4044.FE": "Population ages 40-44, female",
                      "SP.POP.4044.MA": "Population ages 40-44, male",
                      "SP.POP.4549.FE": "Population ages 45-49, female",
                      "SP.POP.4549.MA": "Population ages 45-49, male",
                      "SP.POP.5054.FE": "Population ages 50-54, female",
                      "SP.POP.5054.MA": "Population ages 50-54, male",
                      "SP.POP.5559.FE": "Population ages 55-59, female",
                      "SP.POP.5559.MA": "Population ages 55-59, male",
                      "SP.POP.6064.FE": "Population ages 60-64, female",
                      "SP.POP.6064.MA": "Population ages 60-64, male",
                      "SP.POP.6569.FE": "Population ages 65-69, female",
                      "SP.POP.6569.MA": "Population ages 65-69, male",
                      "SP.POP.65UP.FE.IN": "Population ages 65 and above, female",
                      "SP.POP.65UP.MA.IN": "Population ages 65 and above, male",
                      "SP.POP.65UP.TO": "Population ages 65 and above, total",
                      "SP.POP.7074.FE": "Population ages 70-74, female",
                      "SP.POP.7074.MA": "Population ages 70-74, male",
                      "SP.POP.7579.FE": "Population ages 75-79, female",
                      "SP.POP.7579.MA": "Population ages 75-79, male",
                      "SP.POP.80UP.FE": "Population ages 80 and above, female",
                      "SP.POP.80UP.MA": "Population ages 80 and above, male",

                     # general some environmental variables first
                    "AG.PRD.FOOD.XD": "Food Production Index",
                    "EN.POP.DNST": "Population per sq km", 
                    "SN.ITK.DEFC.ZS": "Prevalence of Undernourishment (% of Population)", 
                    "EG.ELC.COAL.ZS": "Electricity production from coal sources", 
                    "SM.POP.NETM": "Net Migration",
                    "EN.H2O.BDYS.ZS": "Proportion of bodies of water with good ambient water quality", 
                    "ER.H2O.FWTL.ZS": "Annual freshwater withdrawals, total (% of internal resources)",
                    "SH.H2O.SMDW.ZS": "People using safely managed drinking water services (% of population)", 
                    "SH.STA.SMSS.ZS": "People using safely managed sanitation services (% of population)", 
                    "SH.MED.BEDS.ZS": "Hospital beds (per 1,000 people)",
                    
                    # more scientific environmental variables
                    "EN.LND.LTMP.DC": "Land Surface Temperature",
                    "EN.ATM.PM25.MC.M3": "PM2.5 air pollution, mean annual exposure (micrograms per cubic meter)", 
                    "EN.CLC.CDDY.XD": "Cooling Degree Days", 
                    "EN.CLC.HEAT.XD": "Heat Index 35",
                  
                    # some political variables
                      "PV.EST": "Political Stability and Absence of Violence/Terrorism (estimate)", 
                    "IC.LGL.CRED.XQ": "Strength of Legal Rights Index (0-12 Scale)", 
                    "EG.ELC.ACCS.ZS": "Access to electricity (% of population)", 
                    "GE.EST": "Government Effectiveness Estimate", 
                    "SE.XPD.TOTL.GB.ZS": "Government Expenditure on Education (% of Total Expenditure)", 
                    "CC.EST": "Control of Corruption: Estimate", 
                    "SI.DST.FRST.20": "Income Share Held by Lowest 20%", 
                    "SD.ESR.PERF.XQ": "Economic and Social Rights Performance Score",
                    "RL.EST": "Rule of Law: Estimate"} 

In [9]:
# check that function works

population(2015, "Male", 65, 69, "CUW")

3551

In [16]:
# do "unit-test" -- obviously the population is not > 5000, so this should raise an Assertion Error. 
# TLDR: THIS ERROR IS WHAT WE WANT (means both function and unit test are working)

assert population(2015,"Male", 65, 69, 'CUW') > 5000

AssertionError: 

In [80]:
# testing population dataframe function

population_dataframe(2016, "CUW", population_indicators)

,Total Population,"Population, female","Population, male","Population ages 00-04, female","Population ages 00-04, male","Population ages 0-14, female","Population ages 0-14, male","Population ages 0-14, total","Population ages 05-09, female","Population ages 05-09, male",...,"Population ages 65-69, male","Population ages 65 and above, female","Population ages 65 and above, male","Population ages 65 and above, total","Population ages 70-74, female","Population ages 70-74, male","Population ages 75-79, female","Population ages 75-79, male","Population ages 80 and above, female","Population ages 80 and above, male"
date,,,,,,,,,,,,,,,,,,,,,
2016,159664.0,85884.0,73780.0,4829.0,5044.0,15953.0,16623.0,32575.0,5316.0,5539.0,...,3634.0,13145.0,8858.0,22003.0,3616.0,2559.0,2435.0,1563.0,2383.0,1101.0


In [18]:
indicators = wbdata.get_source()
indicators

  id  name
----  --------------------------------------------------------------------
   1  Doing Business
   2  World Development Indicators
   3  Worldwide Governance Indicators
   5  Subnational Malnutrition Database
   6  International Debt Statistics
  11  Africa Development Indicators
  12  Education Statistics
  13  Enterprise Surveys
  14  Gender Statistics
  15  Global Economic Monitor
  16  Health Nutrition and Population Statistics
  18  IDA Results Measurement System
  19  Millennium Development Goals
  20  Quarterly Public Sector Debt
  22  Quarterly External Debt Statistics SDDS
  23  Quarterly External Debt Statistics GDDS
  25  Jobs
  27  Global Economic Prospects
  28  Global Financial Inclusion
  29  The Atlas of Social Protection: Indicators of Resilience and Equity
  30  Exporter Dynamics Database – Indicators at Country-Year Level
  31  Country Policy and Institutional Assessment
  32  Global Financial Development
  33  G20 Financial Inclusion Indicators
  34  Glob